# Лабораторная работа 3 (DecisionTree)

# Импорт всех необходимых модулей

In [1]:
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
    LabelEncoder,
    StandardScaler,
    OneHotEncoder,
    OrdinalEncoder,
)
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
)
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier

warnings.filterwarnings("ignore")


def regression_cross_validate(model_cls, X, y, n_folds: int = 5, **model_kwargs):
    X = np.asarray(X)
    y = np.asarray(y)

    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

    mae_values = []
    mse_values = []
    r2_values = []

    for fold_idx, (idx_train, idx_valid) in enumerate(kf.split(X), start=1):
        X_train, X_valid = X[idx_train], X[idx_valid]
        y_train, y_valid = y[idx_train], y[idx_valid]

        model = model_cls(**model_kwargs)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_valid)

        mae_values.append(mean_absolute_error(y_valid, y_pred))
        mse_values.append(mean_squared_error(y_valid, y_pred))
        r2_values.append(r2_score(y_valid, y_pred))

    metrics_avg = {
        "MAE": float(np.mean(mae_values)),
        "MSE": float(np.mean(mse_values)),
        "R2": float(np.mean(r2_values)),
    }

    metrics_std = {
        "MAE": float(np.std(mae_values)),
        "MSE": float(np.std(mse_values)),
        "R2": float(np.std(r2_values)),
    }

    return metrics_avg, metrics_std


def classification_cross_validate(model_cls, X, y, n_folds: int = 5, **model_kwargs):
    X = np.asarray(X)
    y = np.asarray(y)

    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

    acc_list = []
    prec_list = []
    rec_list = []
    f1_list = []

    for fold_idx, (idx_train, idx_valid) in enumerate(kf.split(X), start=1):
        X_train, X_valid = X[idx_train], X[idx_valid]
        y_train, y_valid = y[idx_train], y[idx_valid]

        model = model_cls(**model_kwargs)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_valid)

        acc_list.append(accuracy_score(y_valid, y_pred))
        prec_list.append(precision_score(y_valid, y_pred, average="weighted", zero_division=0))
        rec_list.append(recall_score(y_valid, y_pred, average="weighted", zero_division=0))
        f1_list.append(f1_score(y_valid, y_pred, average="weighted", zero_division=0))

    metrics_avg = {
        "Accuracy": float(np.mean(acc_list)),
        "Precision": float(np.mean(prec_list)),
        "Recall": float(np.mean(rec_list)),
        "F1-score": float(np.mean(f1_list)),
    }

    metrics_std = {
        "Accuracy": float(np.std(acc_list)),
        "Precision": float(np.std(prec_list)),
        "Recall": float(np.std(rec_list)),
        "F1-score": float(np.std(f1_list)),
    }

    return metrics_avg, metrics_std


# Часть 1. Регрессия
Данные о зарплатах загружаются в DataFrame, и выводятся первые строки, чтобы убедиться, что файл прочитан корректно.

In [2]:
salary_df = pd.read_csv("data/Salary Data.csv")
print("Размер датасета Salary_Data:", salary_df.shape)

salary_df.head()

Размер датасета Salary_Data: (375, 6)


,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0
1,28.0,Female,Master's,Data Analyst,3.0,65000.0
2,45.0,Male,PhD,Senior Manager,15.0,150000.0
3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0
4,52.0,Male,Master's,Director,20.0,200000.0


# Подготовка данных для регрессии

Сначала из датасета убирается столбец `Job Title`: он содержит много уникальных значений и в таком виде мало помогает линейной модели, только раздувает пространство признаков

Категориальные признаки (`Gender`, `Education Level`) переводятся в числовой вид с помощью `OrdinalEncoder`, чтобы их можно было использовать в линейной регрессии и в собственной реализации модели

После этого все пропуски в данных заполняются наиболее частыми значениями (`SimpleImputer` с стратегией `"most_frequent"`). Такой шаг нужен, потому что большинство моделей из `sklearn`, а также наши собственные реализации, не умеют работать с `NaN` и ожидают полностью числовую матрицу признаков без пропусков


In [3]:
reg_df = salary_df.copy()
reg_df = reg_df.dropna(subset=["Salary"])

if "Job Title" in reg_df.columns:
    reg_df = reg_df.drop(columns=["Job Title"])

X_reg = reg_df.drop(columns=["Salary"])
y_reg = reg_df["Salary"]

X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

reg_categorical_cols = ["Gender", "Education Level"]
reg_numeric_cols = [col for col in X_reg.columns if col not in reg_categorical_cols]

reg_ordinal_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

X_reg_train_base = X_reg_train.copy()
X_reg_test_base = X_reg_test.copy()

X_reg_train_base[reg_categorical_cols] = reg_ordinal_encoder.fit_transform(
    X_reg_train_base[reg_categorical_cols]
)
X_reg_test_base[reg_categorical_cols] = reg_ordinal_encoder.transform(
    X_reg_test_base[reg_categorical_cols]
)

reg_imputer = SimpleImputer(strategy="most_frequent")

X_reg_train_base = pd.DataFrame(
    reg_imputer.fit_transform(X_reg_train_base),
    columns=X_reg_train_base.columns
)
X_reg_test_base = pd.DataFrame(
    reg_imputer.transform(X_reg_test_base),
    columns=X_reg_test_base.columns
)

print("NaN в X_train_base:", X_reg_train_base.isna().sum().sum())
print("NaN в X_test_base:", X_reg_test_base.isna().sum().sum())


NaN в X_train_base: 0
NaN в X_test_base: 0


# Построение бейзлайна 

Выполняется кросс-валидация базовой модели решающего дерева для задачи регрессии и выводятся средние значения метрик на обучающих разбиениях. Затем модель обучается на тренировочной выборке, оценивается на тестовой и печатаются итоговые значения

In [4]:
reg_tree_mean, reg_tree_std = regression_cross_validate(
    DecisionTreeRegressor,
    X_reg_train_base.to_numpy(),
    y_reg_train.to_numpy(),
    n_folds=5,
    random_state=42,
)

print("=== Регрессия: бейзлайн (train CV) ===")
print(f"MSE_mean: {reg_tree_mean['MSE']:.2f}")
print(f"MAE_mean: {reg_tree_mean['MAE']:.2f}")
print(f"R2_mean:  {reg_tree_mean['R2']:.3f}")

dt_baseline = DecisionTreeRegressor(random_state=42)
dt_baseline.fit(X_reg_train_base, y_reg_train)

y_reg_pred_tree = dt_baseline.predict(X_reg_test_base)

mse_tree = mean_squared_error(y_reg_test, y_reg_pred_tree)
mae_tree = mean_absolute_error(y_reg_test, y_reg_pred_tree)
r2_tree = r2_score(y_reg_test, y_reg_pred_tree)

print("\nРегрессия - бейзлайн (test)")
print("---------------------------")
print(f"MSE: {mse_tree:.2f}")
print(f"MAE: {mae_tree:.2f}")
print(f"R^2: {r2_tree:.3f}")


=== Регрессия: бейзлайн (train CV) ===
MSE_mean: 356229547.76
MAE_mean: 11227.38
R2_mean:  0.842

Регрессия - бейзлайн (test)
---------------------------
MSE: 231101568.41
MAE: 9428.57
R^2: 0.904


Категориальные признаки кодируются при помощи OneHotEncoder и объединяются с числовыми столбцами в общую таблицу признаков. Затем числовые признаки масштабируются через StandardScaler, чтобы дерево решений работало на более однородных по масштабу данных

In [5]:
reg_onehot = OneHotEncoder(
    sparse_output=False,
    drop="first",
    handle_unknown="ignore",
)

reg_cat_train_ohe = reg_onehot.fit_transform(X_reg_train[reg_categorical_cols])
reg_cat_test_ohe = reg_onehot.transform(X_reg_test[reg_categorical_cols])

reg_ohe_cols = reg_onehot.get_feature_names_out(reg_categorical_cols)

X_reg_train_ohe = pd.concat(
    [
        X_reg_train[reg_numeric_cols].reset_index(drop=True),
        pd.DataFrame(reg_cat_train_ohe, columns=reg_ohe_cols),
    ],
    axis=1,
)

X_reg_test_ohe = pd.concat(
    [
        X_reg_test[reg_numeric_cols].reset_index(drop=True),
        pd.DataFrame(reg_cat_test_ohe, columns=reg_ohe_cols),
    ],
    axis=1,
)

reg_scaler = StandardScaler()
X_reg_train_scaled = X_reg_train_ohe.copy()
X_reg_test_scaled = X_reg_test_ohe.copy()

X_reg_train_scaled[reg_numeric_cols] = reg_scaler.fit_transform(
    X_reg_train_ohe[reg_numeric_cols]
)
X_reg_test_scaled[reg_numeric_cols] = reg_scaler.transform(
    X_reg_test_ohe[reg_numeric_cols]
)


Проводится кросс-валидация модели дерева решений на улучшенных признаках: после one-hot кодирования и масштабирования, с ограничением глубины до 5 уровней. Затем эта модель обучается на всём train-наборе, тестируется на отложенных данных и выводятся обновлённые значения

In [6]:
reg_tree_imp_mean, reg_tree_imp_std = regression_cross_validate(
    DecisionTreeRegressor,
    X_reg_train_scaled.to_numpy(),
    y_reg_train.to_numpy(),
    n_folds=5,
    max_depth=5,
    random_state=42,
)

print("=== Регрессия: улучшенный бейзлайн (train CV) ===")
print(f"MSE_mean: {reg_tree_imp_mean['MSE']:.2f}")
print(f"MAE_mean: {reg_tree_imp_mean['MAE']:.2f}")
print(f"R2_mean:  {reg_tree_imp_mean['R2']:.3f}")

dt_improved = DecisionTreeRegressor(max_depth=5, random_state=42)
dt_improved.fit(X_reg_train_scaled, y_reg_train)

y_reg_pred_imp = dt_improved.predict(X_reg_test_scaled)

mse_imp = mean_squared_error(y_reg_test, y_reg_pred_imp)
mae_imp = mean_absolute_error(y_reg_test, y_reg_pred_imp)
r2_imp = r2_score(y_reg_test, y_reg_pred_imp)

print("\nРегрессия - улучшенный бейзлайн (test)")
print("--------------------------------------")
print(f"MSE: {mse_imp:.2f}")
print(f"MAE: {mae_imp:.2f}")
print(f"R^2: {r2_imp:.3f}")


=== Регрессия: улучшенный бейзлайн (train CV) ===
MSE_mean: 279004099.25
MAE_mean: 11541.22
R2_mean:  0.878

Регрессия - улучшенный бейзлайн (test)
--------------------------------------
MSE: 319154446.65
MAE: 10917.80
R^2: 0.867


# Реализация своего класса

Реализуется собственный регрессор на основе решающего дерева, который рекурсивно строит дерево разбиений по признакам, минимизируя сумму дисперсий в дочерних узлах. В методе predict каждый объект проходит по узлам дерева в зависимости от порогов признаков, пока не достигнет листа с предсказанным средним значением целевой переменной

In [7]:
class MyDecisionTreeRegressor:
    def __init__(self, max_depth: int | None = None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=float)
        self.tree = self._build_tree(X, y, depth=0)
        return self

    def _build_tree(self, X, y, depth: int):
        if len(np.unique(y)) == 1 or (self.max_depth is not None and depth >= self.max_depth):
            return float(np.mean(y))

        best_split = self._find_best_split(X, y)
        if best_split is None:
            return float(np.mean(y))

        feature_idx = best_split["feature"]
        threshold = best_split["value"]

        left_mask = X[:, feature_idx] <= threshold
        right_mask = ~left_mask

        left_subtree = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        right_subtree = self._build_tree(X[right_mask], y[right_mask], depth + 1)

        return {
            "feature": feature_idx,
            "value": threshold,
            "left": left_subtree,
            "right": right_subtree,
        }

    def _find_best_split(self, X, y):
        best_split = None
        best_score = float("inf")

        n_features = X.shape[1]

        for feature in range(n_features):
            values = np.unique(X[:, feature])

            for val in values:
                left_mask = X[:, feature] <= val
                right_mask = ~left_mask

                if left_mask.sum() == 0 or right_mask.sum() == 0:
                    continue

                left_y = y[left_mask]
                right_y = y[right_mask]

                score = self._calculate_split_score(left_y, right_y)

                if score < best_score:
                    best_score = score
                    best_split = {"feature": feature, "value": val}

        return best_split

    @staticmethod
    def _calculate_split_score(left_y, right_y):
        left_score = np.var(left_y) * len(left_y)
        right_score = np.var(right_y) * len(right_y)
        return left_score + right_score

    def predict(self, X):
        X = np.asarray(X, dtype=float)
        return np.array([self._predict_sample(x, self.tree) for x in X])

    def _predict_sample(self, x, node):
        if isinstance(node, dict):
            if x[node["feature"]] <= node["value"]:
                return self._predict_sample(x, node["left"])
            else:
                return self._predict_sample(x, node["right"])
        return node


Проводится k-fold кросс-валидация собственной реализации дерева решений на базовых признаках, после чего выводятся усреднённые метрики качества. Далее самописная модель обучается на train-выборке, тестируется на отложенных данных и по результатам считаются метрики

In [8]:
reg_my_base_mean, reg_my_base_std = regression_cross_validate(
    MyDecisionTreeRegressor,
    X_reg_train_base.to_numpy(),
    y_reg_train.to_numpy(),
    n_folds=5,
)

print("=== Регрессия: собственная модель (train CV, без улучшений) ===")
print(f"MSE_mean: {reg_my_base_mean['MSE']:.2f}")
print(f"MAE_mean: {reg_my_base_mean['MAE']:.2f}")
print(f"R2_mean:  {reg_my_base_mean['R2']:.3f}")

my_tree_base = MyDecisionTreeRegressor()
my_tree_base.fit(X_reg_train_base.to_numpy(), y_reg_train.to_numpy())

y_reg_pred_my_base = my_tree_base.predict(X_reg_test_base.to_numpy())

mse_my_base = mean_squared_error(y_reg_test, y_reg_pred_my_base)
mae_my_base = mean_absolute_error(y_reg_test, y_reg_pred_my_base)
r2_my_base = r2_score(y_reg_test, y_reg_pred_my_base)

print("\nРегрессия - собственная реализация (test, без улучшений)")
print("--------------------------------------------------------")
print(f"MSE: {mse_my_base:.2f}")
print(f"MAE: {mae_my_base:.2f}")
print(f"R^2: {r2_my_base:.3f}")


=== Регрессия: собственная модель (train CV, без улучшений) ===
MSE_mean: 359695139.41
MAE_mean: 11405.86
R2_mean:  0.841

Регрессия - собственная реализация (test, без улучшений)
--------------------------------------------------------
MSE: 260434901.74
MAE: 9961.90
R^2: 0.891


Здесь та же самописная модель дерева решений обучается уже на улучшенных признаках: после one-hot кодирования, масштабирования и с ограничением глубины. Кросс-валидация и тест на отложенной выборке позволяют сравнить, как изменение признаков и параметра max_depth влияет на MSE, MAE и R^2 по сравнению с базовым вариантом

In [9]:
reg_my_imp_mean, reg_my_imp_std = regression_cross_validate(
    MyDecisionTreeRegressor,
    X_reg_train_scaled.to_numpy(),
    y_reg_train.to_numpy(),
    n_folds=5,
    max_depth=5,
)

print("\n=== Регрессия: собственная реализация (train CV, с улучшениями) ===")
print(f"MSE_mean: {reg_my_imp_mean['MSE']:.2f}")
print(f"MAE_mean: {reg_my_imp_mean['MAE']:.2f}")
print(f"R2_mean:  {reg_my_imp_mean['R2']:.3f}")

my_tree_improved = MyDecisionTreeRegressor(max_depth=5)
my_tree_improved.fit(X_reg_train_scaled.to_numpy(), y_reg_train.to_numpy())

y_reg_pred_my_imp = my_tree_improved.predict(X_reg_test_scaled.to_numpy())

mse_my_imp = mean_squared_error(y_reg_test, y_reg_pred_my_imp)
mae_my_imp = mean_absolute_error(y_reg_test, y_reg_pred_my_imp)
r2_my_imp = r2_score(y_reg_test, y_reg_pred_my_imp)

print("\nРегрессия — собственная реализация (test, с улучшениями)")
print("--------------------------------------------------------")
print(f"MSE: {mse_my_imp:.2f}")
print(f"MAE: {mae_my_imp:.2f}")
print(f"R^2: {r2_my_imp:.3f}")



=== Регрессия: собственная реализация (train CV, с улучшениями) ===
MSE_mean: 304992473.79
MAE_mean: 11863.85
R2_mean:  0.866

Регрессия — собственная реализация (test, с улучшениями)
--------------------------------------------------------
MSE: 329006298.51
MAE: 11228.91
R^2: 0.863


# Часть 2. Классификация

# Обработка данных

In [10]:
clf_df = pd.read_csv('data/Student Depression Dataset.csv')
clf_df.head()

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2,Male,33.0,Visakhapatnam,Student,5.0,0.0,8.97,2.0,0.0,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
1,8,Female,24.0,Bangalore,Student,2.0,0.0,5.90,5.0,0.0,5-6 hours,Moderate,BSc,No,3.0,2.0,Yes,0
2,26,Male,31.0,Srinagar,Student,3.0,0.0,7.03,5.0,0.0,Less than 5 hours,Healthy,BA,No,9.0,1.0,Yes,0
3,30,Female,28.0,Varanasi,Student,3.0,0.0,5.59,2.0,0.0,7-8 hours,Moderate,BCA,Yes,4.0,5.0,Yes,1
4,32,Female,25.0,Jaipur,Student,4.0,0.0,8.13,3.0,0.0,5-6 hours,Moderate,M.Tech,Yes,1.0,1.0,No,0


Выполняется разбиение данных по депрессии на обучающую и тестовую выборки с сохранением исходного распределения классов. Затем для всех признаков заполняются пропуски наиболее частыми значениями, чтобы подготовить данные к дальнейшему кодированию и обучению дерева решений

In [11]:
clf_df = clf_df.copy()

X_clf = clf_df.drop(columns=["Depression", "id"])
y_clf = clf_df["Depression"]

X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(
    X_clf,
    y_clf,
    test_size=0.2,
    random_state=42,
    stratify=y_clf,
)

clf_categorical_cols = [
    "Gender",
    "City",
    "Profession",
    "Sleep Duration",
    "Dietary Habits",
    "Degree",
    "Have you ever had suicidal thoughts ?",
    "Family History of Mental Illness",
]

clf_numeric_cols = [
    "Age",
    "Academic Pressure",
    "Work Pressure",
    "CGPA",
    "Study Satisfaction",
    "Job Satisfaction",
    "Work/Study Hours",
    "Financial Stress",
]

clf_imputer = SimpleImputer(strategy="most_frequent")

X_clf_train_imp = pd.DataFrame(
    clf_imputer.fit_transform(X_clf_train),
    columns=X_clf_train.columns,
)
X_clf_test_imp = pd.DataFrame(
    clf_imputer.transform(X_clf_test),
    columns=X_clf_test.columns,
)

Категориальные признаки кодируются с помощью OrdinalEncoder, при этом неизвестные категории явно отправляются в специальное значение 99. Далее оценивается базовое дерево решений по кросс-валидации и на тестовой выборке, чтобы получить отправную точку по точности, полноте и F1-мере для задачи классификации депрессии

In [12]:
clf_ordinal_encoder = OrdinalEncoder(
    handle_unknown="use_encoded_value",
    unknown_value=99,
)

X_clf_train_base = X_clf_train_imp.copy()
X_clf_test_base = X_clf_test_imp.copy()

X_clf_train_base[clf_categorical_cols] = clf_ordinal_encoder.fit_transform(
    X_clf_train_base[clf_categorical_cols]
)
X_clf_test_base[clf_categorical_cols] = clf_ordinal_encoder.transform(
    X_clf_test_base[clf_categorical_cols]
)

print("NaN в X_clf_train_base:", X_clf_train_base.isna().sum().sum())
print("NaN в X_clf_test_base:", X_clf_test_base.isna().sum().sum())

clf_tree_base_mean, clf_tree_base_std = classification_cross_validate(
    DecisionTreeClassifier,
    X_clf_train_base.to_numpy(),
    y_clf_train.to_numpy(),
    n_folds=5,
    random_state=42,
)

print("\n=== Классификация: бейзлайн (train CV) ===")
print(f"Accuracy_mean:  {clf_tree_base_mean['Accuracy']:.3f}")
print(f"Precision_mean:{clf_tree_base_mean['Precision']:.3f}")
print(f"Recall_mean:   {clf_tree_base_mean['Recall']:.3f}")
print(f"F1_mean:       {clf_tree_base_mean['F1-score']:.3f}")

clf_tree_baseline = DecisionTreeClassifier(random_state=42)
clf_tree_baseline.fit(X_clf_train_base, y_clf_train)

y_clf_pred_base = clf_tree_baseline.predict(X_clf_test_base)

acc_base = accuracy_score(y_clf_test, y_clf_pred_base)
prec_base = precision_score(y_clf_test, y_clf_pred_base, average="weighted", zero_division=0)
rec_base = recall_score(y_clf_test, y_clf_pred_base, average="weighted", zero_division=0)
f1_base = f1_score(y_clf_test, y_clf_pred_base, average="weighted", zero_division=0)

print("\nКлассификация - бейзлайн (test)")
print("--------------------------------")
print(f"1. Accuracy:  {acc_base:.2%}")
print(f"2. Precision: {prec_base:.2%}")
print(f"3. Recall:    {rec_base:.2%}")
print(f"4. F1-score:  {f1_base:.2%}")


NaN в X_clf_train_base: 0
NaN в X_clf_test_base: 0

=== Классификация: бейзлайн (train CV) ===
Accuracy_mean:  0.767
Precision_mean:0.768
Recall_mean:   0.767
F1_mean:       0.767

Классификация - бейзлайн (test)
--------------------------------
1. Accuracy:  76.44%
2. Precision: 76.49%
3. Recall:    76.44%
4. F1-score:  76.46%


In [13]:
clf_onehot = OneHotEncoder(
    sparse_output=False,
    drop="first",
    handle_unknown="ignore",
)

train_cat_ohe = clf_onehot.fit_transform(X_clf_train_imp[clf_categorical_cols])
test_cat_ohe = clf_onehot.transform(X_clf_test_imp[clf_categorical_cols])

ohe_clf_cols = clf_onehot.get_feature_names_out(clf_categorical_cols)

X_clf_train_ohe = pd.concat(
    [
        X_clf_train_imp[clf_numeric_cols].reset_index(drop=True),
        pd.DataFrame(train_cat_ohe, columns=ohe_clf_cols),
    ],
    axis=1,
)

X_clf_test_ohe = pd.concat(
    [
        X_clf_test_imp[clf_numeric_cols].reset_index(drop=True),
        pd.DataFrame(test_cat_ohe, columns=ohe_clf_cols),
    ],
    axis=1,
)

clf_scaler = StandardScaler()
X_clf_train_upd = X_clf_train_ohe.copy()
X_clf_test_upd = X_clf_test_ohe.copy()

X_clf_train_upd[clf_numeric_cols] = clf_scaler.fit_transform(
    X_clf_train_ohe[clf_numeric_cols]
)
X_clf_test_upd[clf_numeric_cols] = clf_scaler.transform(
    X_clf_test_ohe[clf_numeric_cols]
)


Для улучшенного дерева решений используем расширенный препроцессинг (One-Hot Encoding + масштабирование числовых признаков) и ограничиваем глубину дерева, чтобы снизить переобучение. Затем оцениваем такую модель с помощью кросс-валидации и на тестовой выборке, сравнивая прирост по Accuracy, Precision, Recall и F1 относительно базовой версии

In [14]:
clf_tree_imp_mean, clf_tree_imp_std = classification_cross_validate(
    DecisionTreeClassifier,
    X_clf_train_upd.to_numpy(),
    y_clf_train.to_numpy(),
    n_folds=5,
    max_depth=5,
    random_state=42,
)

print("=== Классификация: улучшенный бейзлайн (train CV) ===")
print(f"Accuracy_mean:  {clf_tree_imp_mean['Accuracy']:.3f}")
print(f"Precision_mean:{clf_tree_imp_mean['Precision']:.3f}")
print(f"Recall_mean:   {clf_tree_imp_mean['Recall']:.3f}")
print(f"F1_mean:       {clf_tree_imp_mean['F1-score']:.3f}")

clf_tree_improved = DecisionTreeClassifier(max_depth=5, random_state=42)
clf_tree_improved.fit(X_clf_train_upd, y_clf_train)

y_clf_pred_imp = clf_tree_improved.predict(X_clf_test_upd)

acc_imp = accuracy_score(y_clf_test, y_clf_pred_imp)
prec_imp = precision_score(y_clf_test, y_clf_pred_imp, average="weighted", zero_division=0)
rec_imp = recall_score(y_clf_test, y_clf_pred_imp, average="weighted", zero_division=0)
f1_imp = f1_score(y_clf_test, y_clf_pred_imp, average="weighted", zero_division=0)

print("\nКлассификация - улучшенный бейзлайн (test)")
print("------------------------------------------")
print(f"1. Accuracy:  {acc_imp:.2%}")
print(f"2. Precision: {prec_imp:.2%}")
print(f"3. Recall:    {rec_imp:.2%}")
print(f"4. F1-score:  {f1_imp:.2%}")


=== Классификация: улучшенный бейзлайн (train CV) ===
Accuracy_mean:  0.825
Precision_mean:0.825
Recall_mean:   0.825
F1_mean:       0.824

Классификация - улучшенный бейзлайн (test)
------------------------------------------
1. Accuracy:  82.06%
2. Precision: 81.99%
3. Recall:    82.06%
4. F1-score:  81.99%


Реализуем собственный классификатор на основе дерева решений, который строит бинарное дерево, выбирая разбиения по максимуму информационного выигрыша. Для предсказания новый объект спускается по ветвям дерева в зависимости от порогов признаков, пока не достигнет листа с меткой класса

In [15]:
class MyDecisionTreeClassifier:
    def __init__(self, max_depth: int | None = None):
        self.max_depth = max_depth
        self.tree_ = None

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y)
        self.tree_ = self._build_tree(X, y, depth=0)
        return self

    def _build_tree(self, X, y, depth: int):
        n_samples, n_features = X.shape
        classes = np.unique(y)

        if len(classes) == 1:
            return classes[0]
        if n_samples <= 1:
            return self._most_common_class(y)
        if self.max_depth is not None and depth >= self.max_depth:
            return self._most_common_class(y)

        best_split = self._find_best_split(X, y)
        if best_split is None:
            return self._most_common_class(y)

        left_mask = best_split["left_mask"]
        right_mask = best_split["right_mask"]

        left_subtree = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        right_subtree = self._build_tree(X[right_mask], y[right_mask], depth + 1)

        return {
            "feature_idx": best_split["feature_idx"],
            "threshold": best_split["threshold"],
            "left": left_subtree,
            "right": right_subtree,
        }

    def _find_best_split(self, X, y):
        n_samples, n_features = X.shape
        best_gain = -np.inf
        best_split = None

        for feature_idx in range(n_features):
            feature_values = X[:, feature_idx]
            unique_vals = np.unique(feature_values)

            for thr in unique_vals:
                left_mask = feature_values <= thr
                right_mask = feature_values > thr

                if left_mask.sum() == 0 or right_mask.sum() == 0:
                    continue

                gain = self._information_gain(y, left_mask, right_mask)
                if gain > best_gain:
                    best_gain = gain
                    best_split = {
                        "feature_idx": feature_idx,
                        "threshold": thr,
                        "left_mask": left_mask,
                        "right_mask": right_mask,
                    }

        return best_split

    def _information_gain(self, y, left_mask, right_mask):
        y_left = y[left_mask]
        y_right = y[right_mask]

        H_parent = self._entropy(y)
        H_left = self._entropy(y_left)
        H_right = self._entropy(y_right)

        w_left = len(y_left) / len(y)
        w_right = len(y_right) / len(y)

        return H_parent - (w_left * H_left + w_right * H_right)

    @staticmethod
    def _entropy(y):
        classes, counts = np.unique(y, return_counts=True)
        probs = counts / counts.sum()
        return -np.sum(probs * np.log2(probs + 1e-9))

    @staticmethod
    def _most_common_class(y):
        classes, counts = np.unique(y, return_counts=True)
        return classes[np.argmax(counts)]

    def predict(self, X):
        X = np.asarray(X, dtype=float)
        return np.array([self._predict_single(x, self.tree_) for x in X])

    def _predict_single(self, x, node):
        if not isinstance(node, dict):
            return node

        feat_idx = node["feature_idx"]
        thr = node["threshold"]

        if x[feat_idx] <= thr:
            return self._predict_single(x, node["left"])
        else:
            return self._predict_single(x, node["right"])


Для собственной реализации дерева решений проводим K-fold кросс-валидацию и оцениваем средние показатели Accuracy, Precision, Recall и F1 на обучающей выборке. Затем обучаем модель на train-данных и считаем те же метрики на тесте, чтобы сравнить качество с бейзлайном из sklearn

In [16]:
clf_my_base_mean, clf_my_base_std = classification_cross_validate(
    MyDecisionTreeClassifier,
    X_clf_train_base.to_numpy(),
    y_clf_train.to_numpy(),
    n_folds=5,
)

print("=== Классификация: собственная модель (train CV, без улучшений) ===")
print(f"Accuracy_mean:  {clf_my_base_mean['Accuracy']:.3f}")
print(f"Precision_mean:{clf_my_base_mean['Precision']:.3f}")
print(f"Recall_mean:   {clf_my_base_mean['Recall']:.3f}")
print(f"F1_mean:       {clf_my_base_mean['F1-score']:.3f}")

my_tree_base = MyDecisionTreeClassifier()
my_tree_base.fit(X_clf_train_base.to_numpy(), y_clf_train.to_numpy())

y_clf_pred_my_base = my_tree_base.predict(X_clf_test_base.to_numpy())

acc_my_base = accuracy_score(y_clf_test, y_clf_pred_my_base)
prec_my_base = precision_score(y_clf_test, y_clf_pred_my_base, average="weighted", zero_division=0)
rec_my_base = recall_score(y_clf_test, y_clf_pred_my_base, average="weighted", zero_division=0)
f1_my_base = f1_score(y_clf_test, y_clf_pred_my_base, average="weighted", zero_division=0)

print("\nКлассификация - собственная реализация (test, без улучшений)")
print("------------------------------------------------------------")
print(f"1. Accuracy:  {acc_my_base:.2%}")
print(f"2. Precision: {prec_my_base:.2%}")
print(f"3. Recall:    {rec_my_base:.2%}")
print(f"4. F1-score:  {f1_my_base:.2%}")


=== Классификация: собственная модель (train CV, без улучшений) ===
Accuracy_mean:  0.772
Precision_mean:0.773
Recall_mean:   0.772
F1_mean:       0.772

Классификация - собственная реализация (test, без улучшений)
------------------------------------------------------------
1. Accuracy:  76.80%
2. Precision: 76.80%
3. Recall:    76.80%
4. F1-score:  76.80%


Для улучшенной версии собственного дерева решений снова проводим кросс-валидацию, но уже на данных после OneHotEncoding, масштабирования и с ограничением глубины. Затем обучаем модель с `max_depth=5` на этих признаках и считаем Accuracy, Precision, Recall и F1 на тесте, чтобы увидеть, как предобработка и настройка глубины влияют на качество классификации

In [17]:
clf_my_imp_mean, clf_my_imp_std = classification_cross_validate(
    MyDecisionTreeClassifier,
    X_clf_train_upd.to_numpy(),
    y_clf_train.to_numpy(),
    n_folds=5,
    max_depth=5,
)

print("=== Классификация: собственная модель (train CV, с улучшениями) ===")
print(f"Accuracy_mean:  {clf_my_imp_mean['Accuracy']:.3f}")
print(f"Precision_mean:{clf_my_imp_mean['Precision']:.3f}")
print(f"Recall_mean:   {clf_my_imp_mean['Recall']:.3f}")
print(f"F1_mean:       {clf_my_imp_mean['F1-score']:.3f}")

my_tree_improved = MyDecisionTreeClassifier(max_depth=5)
my_tree_improved.fit(X_clf_train_upd.to_numpy(), y_clf_train.to_numpy())

y_clf_pred_my_imp = my_tree_improved.predict(X_clf_test_upd.to_numpy())

acc_my_imp = accuracy_score(y_clf_test, y_clf_pred_my_imp)
prec_my_imp = precision_score(y_clf_test, y_clf_pred_my_imp, average="weighted", zero_division=0)
rec_my_imp = recall_score(y_clf_test, y_clf_pred_my_imp, average="weighted", zero_division=0)
f1_my_imp = f1_score(y_clf_test, y_clf_pred_my_imp, average="weighted", zero_division=0)

print("\nКлассификация - собственная реализация (test, с улучшениями)")
print("------------------------------------------------------------")
print(f"1. Accuracy:  {acc_my_imp:.2%}")
print(f"2. Precision: {prec_my_imp:.2%}")
print(f"3. Recall:    {rec_my_imp:.2%}")
print(f"4. F1-score:  {f1_my_imp:.2%}")


=== Классификация: собственная модель (train CV, с улучшениями) ===
Accuracy_mean:  0.825
Precision_mean:0.825
Recall_mean:   0.825
F1_mean:       0.825

Классификация - собственная реализация (test, с улучшениями)
------------------------------------------------------------
1. Accuracy:  82.06%
2. Precision: 81.99%
3. Recall:    82.06%
4. F1-score:  81.99%


### Заключение

Для регрессии решающие деревья без особого тюнинга дают неплохой результат, но попытки «улучшить» модель (глубина + препроцессинг) местами приводят к лёгкому переобучению на тесте. Для классификации улучшенный препроцессинг и ограничение глубины стабильно поднимают качество, а самописные реализации по метрикам почти полностью повторяют поведение моделей `sklearn`.


### Сводная таблица по регрессии (test)

| Модель                               |    MSE   |    MAE   | R^2 |
| :----------------------------------- | :------: | :------: | :---: |
| Sklearn (до улучшения)               | 2.31e+08 |  9428.57 | 0.904 |
| Sklearn (после улучшения)            | 3.19e+08 | 10917.80 | 0.867 |
| Собственная модель (до улучшения)    | 2.60e+08 |  9961.90 | 0.891 |
| Собственная модель (после улучшения) | 3.29e+08 | 11228.91 | 0.863 |


### Сводная таблица по классификации (test)

| Модель                               | Accuracy | Precision | Recall | F1-score |
| :----------------------------------- | -------: | --------: | -----: | -------: |
| Sklearn (до улучшения)               |   76.44% |    76.49% | 76.44% |   76.46% |
| Sklearn (после улучшения)            |   82.06% |    81.99% | 82.06% |   81.99% |
| Собственная модель (до улучшения)    |   76.80% |    76.80% | 76.80% |   76.80% |
| Собственная модель (после улучшения) |   82.06% |    81.99% | 82.06% |   81.99% |
